# Download articles from PubMed Central

https://www.ncbi.nlm.nih.gov/pmc/tools/oai/

In [1]:
import logging
import zipfile
import tqdm

import lxml.etree
import pandas
import sickle
from pubmedpy.pmc_oai import (
    download_frontmatter_set,
    extract_authors_from_article,
    get_sets_for_pmcid,
)

## Specify PMC-OAI sets

In [2]:
# For each journal in articles.tsv.xz, show two articles and their corresponding sets
pubmed_df = pandas.read_table('data/pubmed/articles.tsv.xz')
pubmed_df = pubmed_df[['pmcid', 'journal']].dropna()
journal_to_n_articles = pubmed_df.journal.value_counts().to_dict()
journal_df = (
    pubmed_df
    .sample(frac=1, random_state=0)
    .groupby('journal').head(2)
    .sort_values(['journal', 'pmcid'])
)
journal_df["oai_set"] = [
    ', '.join(get_sets_for_pmcid(pmcid))
    for pmcid in journal_df.pmcid
]
journal_df

,pmcid,journal,oai_set
5678,PMC2092437,BMC Bioinformatics,"bmcbioi, pmc-open"
12870,PMC3248887,BMC Bioinformatics,"bmcbioi, pmc-open"
8406,PMC2687970,Bioinformatics,"bioinfo, pmc-open"
24181,PMC5860615,Bioinformatics,bioinfo
9666,PMC2816690,PLoS Comput Biol,"ploscomp, pmc-open"
13793,PMC3355066,PLoS Comput Biol,"ploscomp, pmc-open"


In [3]:
# Define journals to download article frontmatter for
journal_to_oai_set = {
    'BMC Bioinformatics': 'bmcbioi',
    'Bioinformatics': 'bioinfo',
    'PLoS Comput Biol': 'ploscomp',
}

## Dowload sets

In [4]:
for journal, oai_set in journal_to_oai_set.items():
    path = f'data/pmc/oai/pmc_fm/{oai_set}.zip'
    download_frontmatter_set(
        oai_set,
        path,
        tqdm=tqdm.tqdm_notebook,
        n_records=journal_to_n_articles[journal],
    )

In [5]:
! ls --size --human-readable data/pmc/oai/pmc_fm

total 54M
11M bioinfo.zip  23M bmcbioi.zip  21M ploscomp.zip
